In [2]:
import numpy as np
import matplotlib.pyplot as plt
import stim
from lib.stabilizer import measurement_gadgets, StabilizerCode, stabilizer_circuits
from lib.color_compass import Lattice2D, compass_to_surface
from lib.decoder import checkmatrix,pL_from_checkmatrix
from lib.stim2pymatching import estimate_pL_noisy_graph
import stimcirq
from typing import *
from cirq.contrib.svg import SVGCircuit

The basic learning loop is as follows 

0) Fix a hidden Pauli noise model for the gates and measurements
1) Pick a compass code and get its stabilizers and logical operators
2) Construct a noisy stim circuit for the code under the noise model 
3) Simulate and get logical error rate
4) Go to 1) and repeat

In [36]:
# Construction of a Pauli noise model

class PauliNoiseModel():
    """
    Constructs noisy Stim circuits from 2D Compass Code stabilizers and logical observables

    A noise model is defined as mapping a perfect operation to an imperfect operation
    """
    def __init__(self, one_qb_gate_rates : List[float] = [0] * 3, two_qb_gate_rates : List[float] = [0] * 15, meas_error_rate : float = 0):
        self.one_qb_gate_rates = one_qb_gate_rates
        assert(len(one_qb_gate_rates) == 3)
        self.two_qb_gate_rates = two_qb_gate_rates
        assert(len(two_qb_gate_rates) == 15)
        self.meas_error_rate = meas_error_rate

    def one_qb_pauli_noise(self) -> str:
        """ 
        Returns a string representing a single qubit Stim Pauli error channel
        """
        channel_str = 'PAULI_CHANNEL_1({},{},{})'.format(*self.one_qb_gate_rates)
        return channel_str 
    
    def two_qb_pauli_noise(self) -> str:
        """ 
        Returns a string representing a two qubit Stim Pauli error channel
        """
        channel_str = 'PAULI_CHANNEL_2({},{},{},{},{},{},{},{},{},{},{},{},{},{},{})'.format(*self.two_qb_gate_rates)
        return channel_str 
    
    def measurement_gadget(self, pauli_observable : str):
        """ 
        Stim gadget to directly measure the specified 'pauli_observable'
        """
        meas_circ = ''
        x_meas_pos = ''
        y_meas_pos = '' 
        z_meas_pos = ''
        pos = {'I' : [], 'X' : [], 'Y' : [], 'Z' : []}
        for i, pauli in enumerate(pauli_observable):
            if pauli == 'X':
                x_meas_pos += f' {i}'
            elif pauli == 'Y':
                y_meas_pos += f' {i}'
            elif pauli == 'Z':
                z_meas_pos += f' {i}'
            else:
                pass 

        meas_circ += f'MX({self.meas_error_rate})' + (x_meas_pos * (len(x_meas_pos) != 0)) + '\n' + f'MY({self.meas_error_rate})' + (y_meas_pos * (len(y_meas_pos) != 0)) + '\n' + f'MZ({self.meas_error_rate})' + (z_meas_pos * (len(z_meas_pos) != 0)) + '\n'
        return stim.Circuit(meas_circ)
    
    def stabilizer_gadget(self, stabilizer_in : str, ancilla_index : int, construction : str = 'cnot'):
        """
        Input:
            stabilizer: a single stabilizer written in terms of {I/_,X,Y,Z}
            construction: direct or hadamard:
                1) `cnot` using only CNOTs from data to ancilla along with single qubit gates
                    - H then S    : rotates Z basis -> Y basis
                    - S_dag then H: rotates Y basis -> Z basis
                    verifiable via checking that: Y stabilizer == kron(S@H,I) @ CNOT @ kron(H@S_dag)
                2) `hadamard` using H gates on ancilla and C-Pauli from ancilla to data
        Output:
            Measurement gadget

        the data qubits that are indicated in the stabilizer appear first
        the ancilla index starts from 0, which is the first ancilla qubit after the data 
        """
         # allow both '_' and 'I' in stabilizers
        stabilizer = stabilizer_in.replace('_','I')
        
        N = len(stabilizer)
        circ_string = ''
        if construction == 'cnot':
            for i, pauli in enumerate(stabilizer):
                if pauli == 'Z':
                    # Z-gates are just cnots from data to ancilla
                    noise_string = self.two_qb_pauli_noise() + f' {i} {N+ancilla_index}\n'
                    circ_string += f'CX {i} {ancilla_index+N} \n' 
                    circ_string += noise_string
                elif pauli == 'X':
                    # X-gates are conjugated by hadamards
                    noise_string_1qb = self.one_qb_pauli_noise() + f' {i}\n'
                    noise_string_2qb = self.two_qb_pauli_noise() + f' {i} {N+ancilla_index}\n'
                    circ_string += f'H {i} \n'
                    circ_string += noise_string_1qb
                    circ_string += f'CX {i} {ancilla_index+N} \n' 
                    circ_string += noise_string_2qb
                    circ_string += f'H {i} \n'
                    circ_string += noise_string_1qb
                elif pauli == 'Y':
                    # Y-gates are conjugated by S-gates and hadamards
                    noise_string_1qb = self.one_qb_pauli_noise() + f' {i}\n'
                    noise_string_2qb = self.two_qb_pauli_noise() + f' {i} {N+ancilla_index}\n'
                    circ_string = f'S_DAG {i} \n'
                    circ_string += noise_string_1qb
                    circ_string += f'H {i} \n'
                    circ_string += noise_string_1qb
                    circ_string += f'CX {i} {ancilla_index+N} \n' 
                    circ_string += noise_string_2qb
                    circ_string += f'H {i} \n'
                    circ_string += noise_string_1qb
                    circ_string += f'S {i} \n'
                    circ_string += noise_string_1qb 

            # noisy ancilla measurement
            circ_string += f'MR({self.meas_error_rate}) {N+ancilla_index}\n'
        return stim.Circuit(circ_string)


    def stabilizer_gadget_v2(self, stabilizer_in : int):
        """
        Use Stim's built in 'MPP' function
        (IS THIS PREFERRED OVER SPLITTING UP MEASUREMENTS INTO CONSTITUENT PARTS AND APPLYING CIRCUIT-LEVEL NOISE?)
        """
        # allow both '_' and 'I' in stabilizers
        stabilizer = stabilizer_in.replace('_','I')
        
        N = len(stabilizer)
        circ_string = f'MPP({self.meas_error_rate}) '
        for i, pauli in enumerate(stabilizer):
            if (pauli != 'I'):
                circ_string += f'{pauli}{i}*'
        circ_string = circ_string[:-1] + '\n'
        return stim.Circuit(circ_string)

In [37]:
"""pick the compass code"""
dim = 9
lat = Lattice2D(dim, dim)

coloring = np.random.randint(-1, 2, size=(dim-1)**2)
lat.color_lattice(coloring)
print(lat)

000---001---002---003---004---005---006---007---008
 |  ▓  |  ░  |  ░  |     |  ▓  |  ▓  |     |     |
009---010---011---012---013---014---015---016---017
 |     |  ▓  |  ░  |     |  ░  |     |  ▓  |  ▓  |
018---019---020---021---022---023---024---025---026
 |     |     |  ▓  |  ░  |  ░  |  ▓  |     |  ▓  |
027---028---029---030---031---032---033---034---035
 |  ▓  |  ▓  |     |  ▓  |     |  ▓  |  ▓  |  ░  |
036---037---038---039---040---041---042---043---044
 |  ░  |  ░  |     |  ▓  |  ▓  |     |     |  ▓  |
045---046---047---048---049---050---051---052---053
 |     |  ▓  |     |     |  ▓  |  ░  |  ▓  |     |
054---055---056---057---058---059---060---061---062
 |  ▓  |     |  ░  |     |  ▓  |     |  ▓  |  ▓  |
063---064---065---066---067---068---069---070---071
 |     |  ░  |  ▓  |     |     |  ░  |     |  ░  |
072---073---074---075---076---077---078---079---080



In [38]:
"""construct the encoding circuit"""
encoding = StabilizerCode(lat.getS()).encoding_circuit(stim=True)
print(encoding)

H 0 1 5 6 9 11 16 17 18 21 24 26 27 28 29 31 33 34 36 40 41 44 45 47 50 52 54 55 59 61 62 63 66
TICK
CX 0 49 1 2 6 7 11 12 21 22 27 39 45 46 50 51 54 65 59 60
TICK
CX 0 73 1 3 6 8 21 23 29 39 45 56
TICK
CX 0 77 1 4 36 39
TICK
CX 0 79 1 49
TICK
CX 0 30 1 14 9 49
TICK
CX 0 20 1 30 5 14 9 73 11 49
TICK
CX 0 68 1 15 9 77 11 13 18 49
TICK
CX 0 69 1 20 6 15 9 79 11 14 18 73 21 49
TICK
CX 0 70 1 68 9 30 18 77 21 42 27 49
TICK
CX 0 71 1 69 9 20 11 30 18 79 21 43 24 42 27 37 31 49
TICK
CX 0 72 1 70 9 68 11 15 18 19 21 53 24 43 27 38 28 37 31 42 36 49
TICK
CX 0 10 1 71 9 69 11 20 18 30 24 53 27 73 29 38 31 43 33 42 36 37 40 49
TICK
CX 0 74 1 10 9 70 11 68 18 20 21 30 27 77 31 32 34 43 36 38 41 42 47 49
TICK
CX 0 75 1 25 9 71 11 69 18 68 27 79 29 30 31 53 36 73 41 43 47 56
TICK
CX 0 76 1 35 6 25 9 72 11 70 18 69 21 68 34 53 36 77 45 73 55 56
TICK
CX 0 78 6 35 9 10 11 71 18 70 21 69 27 68 36 79 41 53 45 77 54 73 55 65
TICK
CX 0 80 9 74 11 25 18 71 21 70 27 69 31 68 44 53 45 79 54 64 55 77 63 65
TI

In [61]:
"""fix a gate set"""
one_qb_rates = [0.01, 0.01, 0.01]
meas_rates = 0.01
pnm = PauliNoiseModel(one_qb_gate_rates=one_qb_rates, meas_error_rate=meas_rates)

In [62]:
"""make a stabilizer gadgets"""
stab_gadget = pnm.stabilizer_gadget(lat.getS()[0], 0)
print(stab_gadget)

H 0
PAULI_CHANNEL_1(0.01, 0.01, 0.01) 0
CX 0 81
PAULI_CHANNEL_2(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 0 81
H 0
PAULI_CHANNEL_1(0.01, 0.01, 0.01) 0
H 9
PAULI_CHANNEL_1(0.01, 0.01, 0.01) 9
CX 9 81
PAULI_CHANNEL_2(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0) 9 81
H 9
PAULI_CHANNEL_1(0.01, 0.01, 0.01) 9
MR(0.01) 81


In [63]:
"""make the observable measurement"""
meas_gadget = pnm.measurement_gadget(lat.Lx)
print(meas_gadget)

MX(0.01) 0 1 2 3 4 5 6 7 8
MY(0.01)
M(0.01)


In [64]:
def compile_compass_circuit(compass_code : Lattice2D, pauli_noise_model : PauliNoiseModel, rounds : int):
    """ 
    We compile a compass code lattice into stim circuits with detectors between subsequent stabilizer measurements

    Params:
    * compass_code - Instance of 'Lattice2D' class that defines compass code
    * pauli_noise_model - Instance of 'PauliNoiseModel' that defines Pauli noise model
    * rounds - Number of rounds of stabilizer measurements we look to perform
    """
    
    compass_circuit = stim.Circuit()

    # Perform encoding into logical all-zeros state
    encoding_circ = StabilizerCode(compass_code.getS()).encoding_circuit(stim=True)
    compass_circuit += encoding_circ
    compass_circuit.append("TICK")

    # Add dummy measurements at start of circuit (X stabs)
    num_X_stabs = len(compass_code.getSx())

    for idx, sx in enumerate(compass_code.getSx()):
        compass_circuit += pauli_noise_model.stabilizer_gadget(sx, idx)
    
    compass_circuit.append("TICK")

    # Perform n rounds of stabilizer measurements and add detectors (X stabs)
    for n in range(rounds):
        for idx, sx in enumerate(compass_code.getSx()):
            compass_circuit += pauli_noise_model.stabilizer_gadget(sx, idx)
        for idx, sx in enumerate(compass_code.getSx()):
            compass_circuit += stim.Circuit(f"DETECTOR({idx}, 0, {n + 1}) rec[{-1 - num_X_stabs - idx}] rec[{-1 - 2 * num_X_stabs - idx}]")

    compass_circuit.append("TICK")

    # Add dummy measurements at start of circuit (Z stabs)
    num_Z_stabs = len(compass_code.getSz())

    for idz, sz in enumerate(compass_code.getSz()):
        compass_circuit += pauli_noise_model.stabilizer_gadget(sz, idz)

    compass_circuit.append("TICK")

    # Perform n rounds of stabilizer measurements and add detectors (Z stabs)
    for n in range(rounds):
        for idz, sz in enumerate(compass_code.getSz()):
            compass_circuit += pauli_noise_model.stabilizer_gadget(sz, idz)
        for idz, sz in enumerate(compass_code.getSz()):
            compass_circuit += stim.Circuit(f"DETECTOR({idz}, 1, {n + 1}) rec[{-1 - num_Z_stabs - idz}] rec[{-1 - 2 * num_Z_stabs - idz}]")

    compass_circuit.append("TICK")
    
    return compass_circuit 

In [56]:
circ = compile_compass_circuit(lat, pnm, 3)

In [57]:
circ

stim.Circuit('''
    H 0 1 5 6 9 11 16 17 18 21 24 26 27 28 29 31 33 34 36 40 41 44 45 47 50 52 54 55 59 61 62 63 66
    TICK
    CX 0 49 1 2 6 7 11 12 21 22 27 39 45 46 50 51 54 65 59 60
    TICK
    CX 0 73 1 3 6 8 21 23 29 39 45 56
    TICK
    CX 0 77 1 4 36 39
    TICK
    CX 0 79 1 49
    TICK
    CX 0 30 1 14 9 49
    TICK
    CX 0 20 1 30 5 14 9 73 11 49
    TICK
    CX 0 68 1 15 9 77 11 13 18 49
    TICK
    CX 0 69 1 20 6 15 9 79 11 14 18 73 21 49
    TICK
    CX 0 70 1 68 9 30 18 77 21 42 27 49
    TICK
    CX 0 71 1 69 9 20 11 30 18 79 21 43 24 42 27 37 31 49
    TICK
    CX 0 72 1 70 9 68 11 15 18 19 21 53 24 43 27 38 28 37 31 42 36 49
    TICK
    CX 0 10 1 71 9 69 11 20 18 30 24 53 27 73 29 38 31 43 33 42 36 37 40 49
    TICK
    CX 0 74 1 10 9 70 11 68 18 20 21 30 27 77 31 32 34 43 36 38 41 42 47 49
    TICK
    CX 0 75 1 25 9 71 11 69 18 68 27 79 29 30 31 53 36 73 41 43 47 56
    TICK
    CX 0 76 1 35 6 25 9 72 11 70 18 69 21 68 34 53 36 77 45 73 55 56
    TICK
    CX 

In [130]:
"""fix a gate set"""
one_qb_rates = [0.01] * 3
# two_qb_rates = [0.9] * 15
meas_rates = 0.001
pnm = PauliNoiseModel(one_qb_gate_rates=one_qb_rates, meas_error_rate=meas_rates)

In [131]:
circ = compile_compass_circuit(lat, pnm, 3)

In [138]:
def shot(circuit : stim.Circuit) -> str:
    sample = circuit.compile_sampler().sample(1)[0]
    sample = sample.astype(int)
    print("".join("_1"[e] for e in sample))
    

In [139]:
shot(circ)

__________1__1__1________________________1_1__1_11____1__11__1___1_____1__111__1_11_______11_11_____11__1__111__1_11_______11_11___1111_11__1_111___1_1___111_11__1_111___1_1___111_11__1_111___1_1___111_11__1_111___1_1___
